In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from attention import *
import numpy as np
import pandas as pd
from datasets import load_from_disk, load_dataset
from tqdm import tqdm
from collections import Counter
import json
from tokenizers import models, Tokenizer, trainers, pre_tokenizers, processors, decoders
from transformers import PreTrainedTokenizerFast

In [ ]:
torch.__version__

'2.2.1+cu121'

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [ ]:
ds = load_dataset("roneneldan/TinyStories")
ds.save_to_disk('data')
ds = load_from_disk('data')
ds.set_format(type="torch")

# Tokenization

In [8]:
train_ds = ds['train']
def get_training_corpus():
    for i in range(0, len(train_ds), 1000):
        yield train_ds[i : i + 1000]["text"]

In [9]:
vocab_size = 25000
tok = Tokenizer(models.BPE())
tok.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
special_tokens = ["[EOT]", '[UNK]', '[PAD]']
trainer = trainers.BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens, unk_token='[UNK]')
tok.train_from_iterator(get_training_corpus(), trainer=trainer)

tok.post_processor = processors.ByteLevel(trim_offsets=False)
tok.decoder = decoders.ByteLevel()

Ignored unknown kwargs option unk_token





In [10]:
fast_tok = PreTrainedTokenizerFast(
    tokenizer_object=tok,
    bos_token="[EOT]",
    eos_token="[EOT]",
    pad_token="[PAD]",
    padding_side="left"
)

In [11]:
max_length = 128

def tokenize(x):
    temp_max_len = max_length + 1
    outputs = fast_tok(
        x['text'],
        truncation=True,
        max_length=temp_max_len,
        padding="max_length",
        return_overflowing_tokens=True,
        return_length=True
    )
    input_batch, label_batch = [], []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == temp_max_len:
            input_batch.append(input_ids[:-1].copy())
            label_batch.append(input_ids[1:].copy())
    return {'input_ids': input_batch, 'labels':label_batch}


In [ ]:
tokenized_datasets = ds.map(
    tokenize, batched=True, remove_columns=ds["train"].column_names
)
tokenized_datasets = tokenized_datasets.with_format('torch', device=DEVICE)

In [ ]:
tokenized_datasets.save_to_disk("tokenized_data")

Saving the dataset (1/1 shards): 100%|██████████| 48149/48149 [00:03<00:00, 13945.20 examples/s]


In [12]:
tokenized_datasets = load_from_disk('tokenized_data')

# Attention Layers

In [ ]:

class Attention(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape, model_size=512):
        super().__init__()
        self.d_q= query_shape[-1]
        self.d_k = key_shape[-1]
        self.d_v = value_shape[-1]
        self.model_size = model_size
        self.W_q= nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty((self.d_q, self.model_size))))
        self.W_k= nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty((self.d_k, self.model_size))))
        self.W_v= nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty((self.d_v, self.model_size))))
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, attention_mask=None):
        query_t = torch.matmul(query, self.W_q)
        key_t = torch.matmul(key, self.W_k)
        value_t = torch.matmul(value, self.W_v)
        query_key=torch.matmul(query_t, key_t.transpose(-2,-1))/math.sqrt(self.d_k)

        if attention_mask is not None:
            query_key = query_key.masked_fill(attention_mask.bool(), float('-inf'))
       
        attention = torch.matmul(self.softmax(query_key), value_t)
        return attention


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, query_shape,key_shape,value_shape, head_count, model_size=512):
        super(MultiHeadAttention,self).__init__()
        self.head_count = head_count
        self.model_size = model_size
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.W_O = nn.Parameter(nn.init.xavier_uniform_(torch.empty(self.head_count*self.value_shape[-1],self.model_size)))

        self.heads = nn.ModuleList([
                                    Attention(self.query_shape, self.key_shape, self.value_shape, self.model_size) 
                                    for _ in range(self.head_count)
                                    ])

    def forward(self, query, key, value, attention_mask =None):
        mh_p1=torch.cat([head(query, key, value, attention_mask) for head in self.heads],-1)
        mh_p2 = torch.matmul(mh_p1, self.W_O)
        return mh_p2

In [ ]:
class FeedForwardNetwork(nn.Module):
    def __init__(self,input_dimension,output_dimension,hidden_dimension):
        super(FeedForwardNetwork,self).__init__()
        self.input_dimension = input_dimension
        self.output_dimension = output_dimension
        self.hidden_dimension = hidden_dimension
        self.linear_1 = nn.Linear(self.input_dimension,self.hidden_dimension)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(self.hidden_dimension,self.output_dimension)

    def forward(self,x):
        layer_1 = self.relu(self.linear_1(x))
        layer_2 = self.relu(self.linear_2(layer_1))
        return layer_2

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape,head_count,model_size,ffn_hidden_dimension,dropout_rate):
        super().__init__()
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.head_count = head_count
        self.model_size = model_size
        self.ffn_hidden_dimension = ffn_hidden_dimension
        self.multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.normal_layer_1 = nn.LayerNorm(self.model_size)
        self.normal_layer_2 = nn.LayerNorm(self.model_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.ffn = FeedForwardNetwork(self.model_size,self.model_size,self.ffn_hidden_dimension)
    
    def forward(self,x):
        x_input = x

        multi_head_output = self.multi_head(x_input,x_input,x_input)
        layer_norm_1_output = self.normal_layer_1(x_input + self.dropout(multi_head_output))

        ffn_output = self.ffn(layer_norm_1_output)
        encoder_output = self.normal_layer_2(layer_norm_1_output + self.dropout(ffn_output))
        
        return encoder_output

In [ ]:
class Decoder(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape,head_count,model_size,ffn_hidden_dimension,dropout_rate):
        super().__init__()
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.head_count = head_count
        self.model_size = model_size
        self.ffn_hidden_dimension = ffn_hidden_dimension
        self.multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.masked_multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.normal_layer_1 = nn.LayerNorm(self.model_size)
        self.normal_layer_2 = nn.LayerNorm(self.model_size)
        self.normal_layer_3 = nn.LayerNorm(self.model_size)
        self.ffn = FeedForwardNetwork(self.model_size,self.model_size,
                                         self.ffn_hidden_dimension)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self,x, encoder_output):

        """
        1. Masked Multi-Head Attention: 
        This is mainly used to prevent overlooking for futuristic values or predictions of the output. 
        """
        x_input = x
        decoder_ip_sentence_length = x_input.shape[-2]
        mask = torch.triu(torch.ones((decoder_ip_sentence_length,
                                      decoder_ip_sentence_length),
                                      device=x.device),diagonal=1)

        masked_attention = self.masked_multi_head(x_input,x_input,x_input,attention_mask = mask)
        layer_norm_1_output = self.normal_layer_1(x_input + self.dropout(masked_attention))

        """
        2. Multi-Head Attention: 
        This is mainly used to prevent overlooking for futuristic values or predictions of the output. 
        Query-> Decoder based 
        Key-> From the output of encoder
        Value-> From the output of encoder
        """
        multi_head_attention = self.multi_head(layer_norm_1_output,encoder_output,encoder_output)
        layer_norm_2_output = self.normal_layer_2(layer_norm_1_output + self.dropout(multi_head_attention))

        # 3. Feed-forward Network
        decoder_output = self.normal_layer_3(layer_norm_2_output + self.ffn(layer_norm_2_output))
        return decoder_output


In [21]:
import torch
import torch.nn as nn
import math

class TransformerEmbedding(nn.Module):
    """Token embeddings for the transformer."""
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        nn.init.uniform_(self.embedding.weight, a=-0.1, b=0.1)
        self.embedding_dim = embedding_dim

    def forward(self, x):
        # Scale embeddings by sqrt(d_model)
        return math.sqrt(self.embedding_dim) * self.embedding(x)

class PositionalEncoding(nn.Module):
    """Sinusoidal positional encoding."""
    def __init__(self, embedding_dim, max_len=5000, wavelength=10000.):
        super().__init__()
        # Create positional encoding matrix [1, max_len, embedding_dim]
        pe = torch.zeros(max_len, embedding_dim)
        position = torch.arange(0, max_len).unsqueeze(1)  # [max_len, 1]
        div_term = torch.exp(
            torch.arange(0, embedding_dim, 2) * (-math.log(wavelength) / embedding_dim)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, embedding_dim]
        self.register_buffer('pe', pe)  # not a parameter, moves to device automatically

    def forward(self, x):
        seq_len = x.size(1)  # x is [batch_size, seq_len]
        return self.pe[:, :seq_len, :]

class ModelEmbeddings(nn.Module):
    """Combined token + positional embeddings with dropout."""
    def __init__(self, vocab_size, embedding_dim, dropout_rate=0.1, max_len=5000):
        super().__init__()
        self.token_emb = TransformerEmbedding(vocab_size, embedding_dim)
        self.pos_emb = PositionalEncoding(embedding_dim, max_len=max_len)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # x: [batch_size, seq_len]
        embeddings = self.token_emb(x) + self.pos_emb(x)
        return self.dropout(embeddings)

In [22]:
class EncoderDecoderTransformer(nn.Module):
    def __init__(self, num_layers, source_vocab_size, target_vocab_size, query_shape, key_shape, value_shape, head_count, model_size, ffn_hidden_dimension, dropout_rate):
        super().__init__()
        self.encoder_layers = nn.ModuleList([Encoder(query_shape, 
                                key_shape, 
                                value_shape, 
                                head_count, 
                                model_size, 
                                ffn_hidden_dimension, 
                                dropout_rate) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([Decoder(query_shape, 
                               key_shape, 
                               value_shape, 
                               head_count, 
                               model_size, 
                               ffn_hidden_dimension, 
                               dropout_rate) for _ in range(num_layers)])
        self.embeddings_ip = ModelEmbeddings(source_vocab_size,model_size,dropout_rate)
        self.embeddings_op = ModelEmbeddings(target_vocab_size,model_size,dropout_rate)

        self.ffn = nn.Linear(model_size,target_vocab_size)
    
    def forward(self,input_text,output_text):
        
        ip_embedding = self.embeddings_ip(input_text)
        op_embedding = self.embeddings_ip(output_text)

        encoder_output = ip_embedding
        for encoder in self.encoder_layers:
            encoder_output = encoder(ip_embedding)
        
        decoder_output = op_embedding
        for decoder in self.decoder_layers:
            decoder_output = decoder(op_embedding, encoder_output)

        output = self.ffn(decoder_output)

        return output

In [23]:
model = EncoderDecoderTransformer(num_layers=6, 
                                  source_vocab_size=vocab_size, target_vocab_size=vocab_size, 
                                  query_shape=(128,), key_shape=(128,), value_shape=(128,), 
                                  head_count=4, model_size=128, ffn_hidden_dimension=128, 
                                  dropout_rate=0.01)
model.to(DEVICE)

EncoderDecoderTransformer(
  (encoder_layers): ModuleList(
    (0-5): 6 x Encoder(
      (multi_head): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Attention(
            (softmax): Softmax(dim=-1)
          )
        )
      )
      (normal_layer_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (normal_layer_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.01, inplace=False)
      (ffn): FeedForwardNetwork(
        (linear_1): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
        (linear_2): Linear(in_features=128, out_features=128, bias=True)
      )
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x Decoder(
      (multi_head): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Attention(
            (softmax): Softmax(dim=-1)
          )
        )
      )
      (masked_multi_head): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x 

In [ ]:
# wrap with torch dataloader for training
batch_size = 64
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, drop_last=True, batch_size=batch_size)
valid_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=batch_size)

In [ ]:
# define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn =  torch.nn.CrossEntropyLoss(ignore_index=fast_tok.pad_token_id)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(
    tokenized_datasets['train'], 
    shuffle=True, 
    drop_last=True, 
    batch_size=batch_size
)
valid_dataloader = DataLoader(
    tokenized_datasets['validation'], 
    shuffle=False,
    drop_last=True,
    batch_size=batch_size
)

# Get vocab size from tokenizer
vocab_size = len(fast_tok)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 14,747,560
Trainable parameters: 14,747,560


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=fast_tok.pad_token_id)

# Optional: Learning rate scheduler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

In [ ]:
def train_one_epoch(epoch_num):
    """Training function for one epoch"""
    running_loss = 0.
    num_batches = len(train_dataloader)

    for i, data in tqdm(enumerate(train_dataloader), total=num_batches, desc=f"Epoch {epoch_num} [Train]"):
        inputs = data['input_ids'].to(DEVICE)
        labels = data['labels'].to(DEVICE)
        
        # For causal language modeling with encoder-decoder:
        # This makes the model learn to reconstruct the input
        decoder_input = inputs
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs, decoder_input)
        
        # Reshape for CrossEntropyLoss: (batch * seq_len, vocab_size) and (batch * seq_len)
        loss = loss_fn(outputs.view(-1, vocab_size), labels.view(-1))
        
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()

        if (i + 1) % 100 == 0:
            avg_loss_so_far = running_loss / (i + 1)
            print(f'  Step {i + 1}/{num_batches} - Avg loss: {avg_loss_so_far:.4f}, Current loss: {loss.item():.4f}')

    # Return average loss for the entire epoch
    avg_loss = running_loss / num_batches
    return avg_loss

In [ ]:
def validate():
    """Validation function"""
    running_vloss = 0.0
    num_batches = len(valid_dataloader)
    
    model.eval()
    
    with torch.no_grad():
        for i, vdata in tqdm(enumerate(valid_dataloader), total=num_batches, desc="Validation"):
            vinputs = vdata['input_ids'].to(DEVICE)
            vlabels = vdata['labels'].to(DEVICE)
            
            # Same decoder input strategy as training
            decoder_input = vinputs
            
            voutputs = model(vinputs, decoder_input)
            
            vloss = loss_fn(voutputs.view(-1, vocab_size), vlabels.view(-1))
            running_vloss += vloss.item()
    
    avg_vloss = running_vloss / num_batches
    return avg_vloss


def calculate_perplexity(loss):
    """Calculate perplexity from loss"""
    return torch.exp(torch.tensor(loss)).item()


In [ ]:
EPOCHS = 1 # TinyStories typically needs 2-5 epochs
best_vloss = float('inf')

print(f"\nStarting training on TinyStories dataset...")
print(f"Training samples: {len(tokenized_datasets['train']):,}")
print(f"Validation samples: {len(tokenized_datasets['validation']):,}")
print(f"Batch size: {batch_size}")
print(f"Steps per epoch: {len(train_dataloader):,}\n")

for epoch in range(EPOCHS):
    print(f'\n{"="*70}')
    print(f'EPOCH {epoch + 1}/{EPOCHS}')
    print(f'{"="*70}')

    # Train model
    model.train()
    avg_loss = train_one_epoch(epoch + 1)
    train_perplexity = calculate_perplexity(avg_loss)
    
    print(f'\n✓ Training complete')
    print(f'  Average Loss: {avg_loss:.4f}')
    print(f'  Perplexity: {train_perplexity:.2f}')

    # Validation
    print('\nRunning validation...')
    avg_vloss = validate()
    val_perplexity = calculate_perplexity(avg_vloss)
    
    # Update learning rate scheduler
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f'\n{"="*70}')
    print(f'EPOCH {epoch + 1} SUMMARY:')
    print(f'  Training Loss:       {avg_loss:.4f}  (Perplexity: {train_perplexity:.2f})')
    print(f'  Validation Loss:     {avg_vloss:.4f}  (Perplexity: {val_perplexity:.2f})')
    print(f'  Learning Rate:       {current_lr:.6f}')
    print(f'{"="*70}')
    
    # Save best model
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_loss,
            'val_loss': avg_vloss,
            'train_perplexity': train_perplexity,
            'val_perplexity': val_perplexity,
        }, 'best_tinystories_model.pth')
        print(f'✓ Saved new best model (val_loss: {best_vloss:.4f}, perplexity: {val_perplexity:.2f})')
    
    # Save checkpoint every epoch
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': avg_loss,
        'val_loss': avg_vloss,
    }, f'checkpoint_epoch_{epoch + 1}.pth')



Starting training on TinyStories dataset...
Training samples: 4,741,600
Validation samples: 48,149
Batch size: 64
Steps per epoch: 74,087


EPOCH 1/1


Epoch 1 [Train]:   0%|          | 0/74087 [00:00<?, ?it/s]

Epoch 1 [Train]:   0%|          | 100/74087 [00:33<10:27:05,  1.97it/s]

  Step 100/74087 - Avg loss: 8.8418, Current loss: 7.7956


Epoch 1 [Train]:   0%|          | 200/74087 [01:24<25:20:28,  1.23s/it]

...

  Step 74000/74087 - Avg loss: 0.1324, Current loss: 0.0154


Epoch 1 [Train]: 100%|██████████| 74087/74087 [2:56:53<00:00,  6.98it/s]



✓ Training complete
  Average Loss: 0.1323
  Perplexity: 1.14

Running validation...


Validation: 100%|██████████| 752/752 [00:44<00:00, 17.09it/s]



EPOCH 1 SUMMARY:
  Training Loss:       0.1323  (Perplexity: 1.14)
  Validation Loss:     0.0196  (Perplexity: 1.02)
  Learning Rate:       0.000098
✓ Saved new best model (val_loss: 0.0196, perplexity: 1.02)


In [ ]:
print(f'\nTraining complete! Best validation loss: {best_vloss:.4f}')


Training complete! Best validation loss: 0.0196


In [51]:
checkpoint = torch.load('best_tinystories_model.pth', map_location=DEVICE)


In [32]:

print("Checkpoint keys:", checkpoint.keys())


Checkpoint keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'train_loss', 'val_loss', 'train_perplexity', 'val_perplexity'])


In [52]:

# Load only the model state dict
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(DEVICE)

In [53]:

model.eval()

print("✓ Model loaded successfully!")

# You can also see training info
print(f"Epoch: {checkpoint['epoch']}")
print(f"Train Loss: {checkpoint['train_loss']:.4f}")
print(f"Val Loss: {checkpoint['val_loss']:.4f}")

✓ Model loaded successfully!
Epoch: 1
Train Loss: 0.1323
Val Loss: 0.0196


In [54]:
def generate(text,max_length=50):
    model.eval()
    ids = fast_tok.encode(text,return_tensors='pt').to(DEVICE)
    generated = ids

    with torch.no_grad():
        for _ in range(max_length):
            out = model(generated,generated)
            next_token = torch.argmax(
                out[:,-1,:],dim=-1,keepdim=True
            )
            generated = torch.cat([generated, next_token],dim=1)
            
            if next_token.item() == fast_tok.eos_token_id:
                break
    
    return fast_tok.decode(generated[0],skip_special_tokens=True)

In [71]:
def generate_with_repetition_penalty(text, max_length=50, temperature=0.8, repetition_penalty=1.2):
    """Generate text with repetition penalty"""
    model.eval()
    ids = fast_tok.encode(text, return_tensors='pt').to(DEVICE)
    generated = ids
    
    with torch.no_grad():
        for _ in range(max_length):
            out = model(generated, generated)
            next_token_logits = out[:, -1, :]
            
            # Apply repetition penalty
            for token_id in set(generated[0].tolist()):
                # Penalize tokens that have already been generated
                if next_token_logits[0, token_id] < 0:
                    next_token_logits[0, token_id] *= repetition_penalty
                else:
                    next_token_logits[0, token_id] /= repetition_penalty
            
            # Apply temperature
            next_token_logits = next_token_logits / temperature
            
            # Sample
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            generated = torch.cat([generated, next_token], dim=1)
            
            if next_token.item() == fast_tok.eos_token_id:
                break
    
    return fast_tok.decode(generated[0], skip_special_tokens=True)


# Test
print("\nWith Repetition Penalty:")
result = generate_with_repetition_penalty("Once upon a time", max_length=17, repetition_penalty=2.5)
print(result)


With Repetition Penalty:
Once upon a time today shares raining impossible stuffing special every From pastime Richard Kathy gaspmar mess completing elsebie
